In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import pmdarima as pmd
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE

### Read the data

In [ ]:
df  = pd.read_csv('AAPL.csv')

In [ ]:
df.shape

### Describe 


In [ ]:
df.describe()

In [ ]:
df

In [ ]:
df[df.isna()].sum() # no NA 

In [ ]:
df['Moving_Avg'] = df['Close'].rolling(window=30).mean()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(df['Date'], df['Close'], label = 'Close price')
# plt.plot(df['Date'], df['Moving_Avg'], color='red', label='30-Day Moving Average', alpha=0.7)
plt.gca().xaxis.set_major_locator(mdates.YearLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.grid(True)
plt.xticks(rotation=45)
plt.ylabel('Price in USD')
plt.xlabel('Time')
plt.title("Closing price of AAPL over time.")
plt.legend()
plt.savefig(f"daily_stock_price.pdf", format="pdf", dpi=500, bbox_inches = "tight")
plt.show()

In [ ]:
df['Month_Year'] = df['Date'].dt.to_period('M')


In [ ]:
last_days = df.groupby('Month_Year').tail(1)

# Reset the index if needed
last_days.reset_index(drop=True, inplace=True)
last_days.set_index('Month_Year')

In [ ]:
df = last_days

In [ ]:
df

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(df['Date'], df['Close'], label = 'Close price', )
# plt.plot(df['Date'], df['Moving_Avg'], color='red', label='14-Day Moving Average', alpha= 0.7)
plt.gca().xaxis.set_major_locator(mdates.YearLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.grid(True)
plt.xticks(rotation=45)
plt.ylabel('Price in USD')
plt.xlabel('Time')
plt.title("Closing price of AAPL over time.")
plt.legend()
plt.savefig(f"last_days_price.pdf", format="pdf", dpi=500, bbox_inches = "tight")
plt.show()

In [ ]:
df = df[df['Date'] >= '2010-01-01'].copy()

df_new = df[df['Date'] >= '2010-01-01'].copy()
# df_new['Moving_Avg'] = df_new['Close'].rolling(window=14).mean()

plt.figure(figsize=(20, 6))
plt.plot(df_new['Date'], df_new['Close'], label='Close Price')
# plt.plot(df_new['Date'], df_new['Moving_Avg'], color='red', label='30-Day Moving Average')
plt.gca().xaxis.set_major_locator(mdates.YearLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

plt.grid(True)
plt.xticks(rotation=45)
plt.ylabel('Price in USD')
plt.xlabel('Time')
plt.title("Closing price of AAPL over time (2010 and newer)")
plt.savefig(f"last_day_stock_price_after_2010.pdf", format="pdf", dpi=500, bbox_inches = "tight")

plt.legend()
plt.show()

In [ ]:
# df = df[df['Date'] >= '2013-01-01']

In [ ]:
df

In [ ]:
df_new.set_index('Date')
res = sm.tsa.seasonal_decompose(df_new['Close'], model='multiplicative', period=12)
res.plot()
plt.savefig(f"decomposition.pdf", format="pdf", dpi=500, bbox_inches = "tight")
plt.show()

In [ ]:
[i for i in range(df.shape[0])]

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

# Extract month as a numerical value and create a new column 'Month'
df['Month'] = df['Date'].dt.month

In [ ]:
X = [i for i in range(df.shape[0])]
y = df['Close']

# Fit the regression model
model = sm.OLS(y, X).fit()

# Step 4: Print the model summary
print(model.summary())

In [ ]:
X = df['Month']
y = df['Close']

# Fit the regression model
model = sm.OLS(y, X).fit()

# Step 4: Print the model summary
print(model.summary())

In [ ]:
res.seasonal.plot()
plt.show()

### Testing ACF and PACF


- ACF decays slowly
- PACF cuts off sharply at lag 1


-> autoregressive process AR(1) 

In [ ]:
df['Close_diff'] = df['Close'] - df['Close'].shift()
df['Close_second_diff'] = df['Close'].diff().diff()
df_train = df[df['Date'] <= '2021-01-01'].copy()
df_train = df_train[df_train['Date'] >= '2010-01-01'].copy()
df_test  = df[df['Date'] >= '2021-01-01'].copy()

In [ ]:
def test_stationarity(df, col):
    result = sm.tsa.adfuller(df[col])
    
    adf_statistic = result[0]
    p_value = result[1]
    used_lag = result[2]
    n_obs = result[3]
    critical_values = result[4]
    ic_best = result[5]
    
    print(f'ADF Statistic: {adf_statistic}')
    print(f'p-value: {p_value}')
    print(f'Used Lag: {used_lag}')
    print(f'Number of Observations Used: {n_obs}')
    print('Critical Values:')
    for key, value in critical_values.items():
        print(f'   {key}: {value}')
    print(f'Best Information Criterion: {ic_best}')
    
    # Interpret the result
    if p_value < 0.05:
        print("The time series is stationary.")
    else:
        print("The time series is not stationary.")

In [ ]:
test_stationarity(df, 'Close')

In [ ]:
df['Close_diff'] = df['Close'].diff()
df.dropna(inplace=True)
test_stationarity(df, 'Close_diff')

In [ ]:
df.dropna(inplace=True)
test_stationarity(df, 'Close_second_diff')

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(df_train['Date'], df_train['Close_second_diff'], label = 'Close price')
plt.gca().xaxis.set_major_locator(mdates.YearLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.grid(True)
plt.xticks(rotation=45)
plt.ylabel('Price increase in USD')
plt.xlabel('Time')
plt.title("Closing price of AAPL over time. Differenced.")
plt.legend()
plt.savefig(f"differenced_2.pdf", format="pdf", dpi=500, bbox_inches = "tight")
plt.show()

In [ ]:
def format_with_spaces(x, pos):
    return f'{x:,.0f}'.replace(',', ' ')

plt.figure(figsize=(20,6))
plt.plot(df_train['Date'], df_train['Volume'], label = 'Close price')
plt.ticklabel_format(style= 'plain', axis = 'y')
plt.gca().xaxis.set_major_locator(mdates.YearLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(format_with_spaces))
plt.grid(True)
plt.xticks(rotation=45)
plt.ylabel('Volume of stocks traded.')
plt.xlabel('Time')
plt.title("Volume of AAPL stocks traded over time.")
plt.legend()
plt.show()

In [ ]:
#Plot ACF
plt.figure(figsize=(10, 6))
sm.graphics.tsa.plot_acf(df_train['Close'].dropna(), lags=15)
plt.title('Autocorrelation Function (ACF) of AAPL Close Prices')
plt.xlabel('Lags')
plt.savefig(f"acf.pdf", format="pdf", dpi=500, bbox_inches = "tight")
plt.show()

# Plot PACF
plt.figure(figsize=(12, 6))
sm.graphics.tsa.plot_pacf(df_train['Close'].dropna(), lags=15)
plt.title('Partial Autocorrelation Function (PACF) of AAPL Close Prices')
plt.xlabel('Lags')
plt.savefig(f"pacf.pdf", format="pdf", dpi=500, bbox_inches = "tight")
plt.show()

In [ ]:
df_train.dropna(inplace=True)
pmd.auto_arima(df_train['Close'],start_p=1,start_q=1,test='adf',m=12,seasonal=True, trace=True)

In [ ]:
sarima=sm.tsa.SARIMAX(df_train['Close'].dropna(),order=(1,2,0),seasonal_order=(0,0,0,12))
model = sarima.fit()
predicted = model.predict()

In [ ]:
residuals = model.resid
plt.figure(figsize=(12, 6))
plt.plot(residuals, label='Residuals')
plt.title('Residuals of SARIMA Model')
plt.legend()
plt.show()

# Plot the squared residuals
plt.figure(figsize=(12, 6))
plt.plot(residuals**2, label='Squared Residuals')
plt.title('Squared Residuals of SARIMA Model')
plt.legend()
plt.show()

In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox

# Perform the Ljung-Box test
result = acorr_ljungbox(residuals)
print(result)


In [ ]:
bp_test = sm.stats.diagnostic.het_breuschpagan(residuals, sm.add_constant(model.fittedvalues))
bp_test_statistic = bp_test[0]
bp_test_pvalue = bp_test[1]

print(f'Breusch-Pagan test statistic: {bp_test_statistic}')
print(f'Breusch-Pagan test p-value: {bp_test_pvalue}')

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(df_train['Date'], df_train['Close'], label = 'Close price', color='blue')
plt.plot(df_train['Date'], predicted, label = 'Prediction', color='yellow')
plt.gca().xaxis.set_major_locator(mdates.YearLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.grid(True)
plt.xticks(rotation=45)
plt.ylabel('Price increase in USD')
plt.xlabel('Time')
plt.title("Closing price of AAPL over time. Differenced.")
plt.legend()
plt.show()
MSE(df_train['Close'], predicted)
forecast = model.forecast(15, exog = None)


In [ ]:
plt.figure(figsize=(20,6))
plt.plot(df_train['Date'], df_train['Close'], label = 'Close price', color='blue', alpha= 0.8)
plt.plot(df_test['Date'], df_test['Close'], color='blue')
plt.plot(df_test['Date'], forecast, label = 'Forecast', color='red')
plt.plot(df_train['Date'], predicted, label = 'Fit on Training data', color='orange', alpha = 0.8)

# plt.plot(df_test['Date'], forecast, label = 'Prediction', color='yellow')
plt.gca().xaxis.set_major_locator(mdates.YearLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.grid(True)
plt.xticks(rotation=45)
plt.ylabel('Price increase in USD')
plt.xlabel('Time')
plt.title("Closing price of AAPL over time. Differenced.")
plt.legend()
plt.show()

print(MSE(df_train['Close'], predicted))
print(MSE(df_test['Close'], forecast))


print(MAE(df_train['Close'], predicted))
print(MAE(df_test['Close'], forecast))

In [ ]:
df['Log_Close'] = np.log(df['Close'])

# Display the first few rows to verify the transformation
print(df.head())

# Plot the original and log-transformed Close price for comparison
plt.figure(figsize=(14, 7))

# Original Close price plot
plt.subplot(2, 1, 1)
plt.plot(df.index, df['Close'], label='Original Close Price')
plt.title('Original Close Price')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.legend()

# Log-transformed Close price plot
plt.subplot(2, 1, 2)
plt.plot(df.index, df['Log_Close'], label='Log-transformed Close Price', color='orange')
plt.title('Log-transformed Close Price')
plt.xlabel('Date')
plt.ylabel('Log(Close Price)')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
df['Close_diff'] = df['Close'] - df['Close'].shift()
df['Log_Close_Diff'] = df['Log_Close'] - df['Log_Close'].shift()
df.dropna(inplace=True)
df_train = df[df['Date'] <= '2021-01-01'].copy()
df_train = df_train[df_train['Date'] >= '2010-01-01'].copy()
df_test  = df[df['Date'] >= '2021-01-01'].copy()

In [ ]:
test_stationarity(df, 'Log_Close')

In [ ]:
df['Log_Close_Diff']

In [ ]:
test_stationarity(df, 'Log_Close_Diff')

In [ ]:
#Plot ACF
plt.figure(figsize=(10, 6))
sm.graphics.tsa.plot_acf(df_train['Log_Close'].dropna(), lags=10)
plt.title('Autocorrelation Function (ACF) of AAPL Close Prices')
plt.xlabel('Lags')
plt.show()

# Plot PACF
plt.figure(figsize=(12, 6))
sm.graphics.tsa.plot_pacf(df_train['Log_Close'].dropna(), lags=10)
plt.title('Partial Autocorrelation Function (PACF) of AAPL Close Prices')
plt.xlabel('Lags')
plt.show()

In [ ]:
sarima=sm.tsa.SARIMAX(df_train['Log_Close'],order=(2,2,0),seasonal_order=(0,1,0,12))
model = sarima.fit()
predicted = model.predict()

In [ ]:
residuals = model.resid[1:]
plt.figure(figsize=(12, 6))
plt.plot(residuals, label='Residuals')
plt.title('Residuals of SARIMA Model')
plt.legend()
plt.show()

# Plot the squared residuals
plt.figure(figsize=(12, 6))
plt.plot(residuals**2, label='Squared Residuals')
plt.title('Squared Residuals of SARIMA Model')
plt.legend()
plt.show()

In [ ]:
result = acorr_ljungbox(residuals)
print(result)

In [ ]:
bp_test = sm.stats.diagnostic.het_breuschpagan(residuals, sm.add_constant(model.fittedvalues)[1:])
bp_test_statistic = bp_test[0]
bp_test_pvalue = bp_test[1]

print(f'Breusch-Pagan test statistic: {bp_test_statistic}')
print(f'Breusch-Pagan test p-value: {bp_test_pvalue}')

In [ ]:
forecast = model.forecast(15, exog = None)
plt.figure(figsize=(20,6))
plt.plot(df['Date'], df['Log_Close'], label = 'Close price', color='blue', alpha= 0.8)
plt.plot(df_test['Date'], df_test['Log_Close'], color='blue')
plt.plot(df_test['Date'], forecast, label = 'Forecast', color='red')
plt.plot(df_train['Date'], predicted, label = 'Fit on Training data', color='orange', alpha = 0.8)

# plt.plot(df_test['Date'], forecast, label = 'Prediction', color='yellow')
plt.gca().xaxis.set_major_locator(mdates.YearLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.grid(True)
plt.xticks(rotation=45)
plt.ylabel('Price increase in USD')
plt.xlabel('Time')
plt.title("Closing price of AAPL over time. Differenced.")
plt.legend()
plt.show()

print(MSE(df_train['Log_Close'], predicted))
print(MSE(df_test['Log_Close'], forecast))


print(MAE(df_train['Log_Close'], predicted))
print(MAE(df_test['Log_Close'], forecast))